## 専門知識労働者
- 保険テクノロジー企業である Insurellm の従業員が使用する、専門知識を持つナレッジ ワーカーである質問応答エージェント。エージェントは正確である必要があり、ソリューションは低コストである必要があります。
- このプロジェクトでは、RAG（検索拡張生成）を使用して、質問/回答アシスタントが高い精度を確保します。この最初の実装では、シンプルでブルートフォース・タイプのRAGが使用されます。

### Sidenote：今週のプロジェクトのビジネスアプリケーション
RAGは、このコースで扱う技術の中で、おそらく最も即戦力となるでしょう。実際、今週構築するのと全く同じ機能、つまり企業契約や製品仕様といった大規模な情報データベース全体にわたるきめ細かなクエリを実行する商用製品が存在します。RAGは、LLMをビジネス分野に適応させるための、市場投入までの期間が短く、低コストなメカニズムを提供します。

In [1]:
# import
import os
import glob # ファイルパス検索
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
# 低コストモデルを使用
MODEL = "gpt-4o-mini"

In [3]:
# .envファイルから環境変数をロード
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
openai = OpenAI()

In [4]:
# 入れ物の初期化
context = {}

## 従業員フォルダのファイルから、従業員情報をディクショナリ化する。

In [5]:
# Windowsユーザーのために以下でバグを修正してくれた学生John S.博士に大いに感謝します！

employees = glob.glob("knowledge-base/employees/*")

for employee in employees:
    name = employee.split(' ')[-1][:-3]
    doc = ""
    with open(employee, "r", encoding="utf-8") as f:
        doc = f.read()
    context[name]=doc

In [6]:
# キーの一覧
keys = context.keys()
print(list(keys))

['Chen', 'Harper', 'Thomson', 'Lancaster', 'Carter', 'Tran', 'Blake', 'Bishop', 'Thompson', 'Spencer', 'Greene', 'Trenton']


In [7]:
# 従業員（Lancaster）の情報
context["Lancaster"]

"# Avery Lancaster\n\n## Summary\n- **Date of Birth**: March 15, 1985  \n- **Job Title**: Co-Founder & Chief Executive Officer (CEO)  \n- **Location**: San Francisco, California  \n\n## Insurellm Career Progression\n- **2015 - Present**: Co-Founder & CEO  \n  Avery Lancaster co-founded Insurellm in 2015 and has since guided the company to its current position as a leading Insurance Tech provider. Avery is known for her innovative leadership strategies and risk management expertise that have catapulted the company into the mainstream insurance market.  \n\n- **2013 - 2015**: Senior Product Manager at Innovate Insurance Solutions  \n  Before launching Insurellm, Avery was a leading Senior Product Manager at Innovate Insurance Solutions, where she developed groundbreaking insurance products aimed at the tech sector.  \n\n- **2010 - 2013**: Business Analyst at Edge Analytics  \n  Prior to joining Innovate, Avery worked as a Business Analyst, focusing on market trends and consumer preferenc

## 製品フォルダのファイルから、製品をディクショナリ化する。

In [8]:
products = glob.glob("knowledge-base/products/*")

for product in products:
    name = product.split(os.sep)[-1][:-3]
    doc = ""
    with open(product, "r", encoding="utf-8") as f:
        doc = f.read()
    context[name]=doc

In [9]:
# キーの一覧
context.keys()

dict_keys(['Chen', 'Harper', 'Thomson', 'Lancaster', 'Carter', 'Tran', 'Blake', 'Bishop', 'Thompson', 'Spencer', 'Greene', 'Trenton', 'Carllm', 'Homellm', 'Markellm', 'Rellm'])

In [10]:
# 製品（Homellm）の情報
context["Homellm"]

"# Product Summary\n\n# Homellm\n\n## Summary\nHomellm is an innovative home insurance product developed by Insurellm that leverages advanced AI technology to revolutionize the way insurance providers offer coverage to homeowners. Designed for both B2B and B2C segments, Homellm empowers insurers to provide personalized, data-driven policies, enhancing customer experience while minimizing risk and operational costs. By integrating seamlessly with existing systems, Homellm helps insurance companies streamline their processes and stay competitive in the ever-evolving insurance industry.\n\n## Features\n### 1. AI-Powered Risk Assessment\nHomellm utilizes sophisticated AI algorithms to analyze vast datasets, allowing insurance companies to assess risks accurately. This feature provides real-time insights for underwriting decisions, enabling insurers to tailor policies to individual customer needs.\n\n### 2. Dynamic Pricing Model\nWith Homellm's innovative dynamic pricing model, insurance pr

In [11]:
# あなたは、保険テクノロジー企業Insurellmに関する正確な質問に答えるエキスパートです。簡潔かつ正確な回答を心がけてください。
# 答えがわからない場合は、その旨を伝えてください。関連する背景情報が提供されていない場合は、勝手に答えないでください。
system_message = "You are an expert in answering accurate questions about Insurellm, the Insurance Tech company.\
Give brief, accurate answers. If you don't know the answer, say so. Do not make anything up if you haven't been provided with relevant context."

In [12]:
# はじめにcontext（辞書型）をキー（context_title）値（context_details）に分割し、
# 与えられた文字列 message の中に、キーと同じ文字列が含まれてい対応する値をリストに集めて返す。
def get_relevant_context(message):
    relevant_context = []
    for context_title, context_details in context.items():
        if context_title.lower() in message.lower():
            relevant_context.append(context_details)
    return relevant_context          

In [13]:
# lancasterがヒット
get_relevant_context("Who is lancaster?")

["# Avery Lancaster\n\n## Summary\n- **Date of Birth**: March 15, 1985  \n- **Job Title**: Co-Founder & Chief Executive Officer (CEO)  \n- **Location**: San Francisco, California  \n\n## Insurellm Career Progression\n- **2015 - Present**: Co-Founder & CEO  \n  Avery Lancaster co-founded Insurellm in 2015 and has since guided the company to its current position as a leading Insurance Tech provider. Avery is known for her innovative leadership strategies and risk management expertise that have catapulted the company into the mainstream insurance market.  \n\n- **2013 - 2015**: Senior Product Manager at Innovate Insurance Solutions  \n  Before launching Insurellm, Avery was a leading Senior Product Manager at Innovate Insurance Solutions, where she developed groundbreaking insurance products aimed at the tech sector.  \n\n- **2010 - 2013**: Business Analyst at Edge Analytics  \n  Prior to joining Innovate, Avery worked as a Business Analyst, focusing on market trends and consumer preferen

In [14]:
# carllmがヒット（Alex Lancaster は Lancaster で登録されているのでヒットしない）
get_relevant_context("Who is Avery and what is carllm?")

['# Product Summary\n\n# Carllm\n\n## Summary\n\nCarllm is an innovative auto insurance product developed by Insurellm, designed to streamline the way insurance companies offer coverage to their customers. Powered by cutting-edge artificial intelligence, Carllm utilizes advanced algorithms to deliver personalized auto insurance solutions, ensuring optimal coverage while minimizing costs. With a robust infrastructure that supports both B2B and B2C customers, Carllm redefines the auto insurance landscape and empowers insurance providers to enhance customer satisfaction and retention.\n\n## Features\n\n- **AI-Powered Risk Assessment**: Carllm leverages artificial intelligence to analyze driver behavior, vehicle conditions, and historical claims data. This enables insurers to make informed decisions and set competitive premiums that reflect true risk profiles.\n\n- **Instant Quoting**: With Carllm, insurance companies can offer near-instant quotes to customers, enhancing the customer exper

In [15]:
# プロンプトを組み立てる（入力された文字列 message に対して、関連する context 情報を追加）
def add_context(message):
    relevant_context = get_relevant_context(message) # 前述の関数
    if relevant_context:
        message += "\n\nThe following additional context might be relevant in answering this question:\n\n"
        for relevant in relevant_context:
            message += relevant + "\n\n"
    return message

In [16]:
print(add_context("Who is Alex Lancaster?"))

Who is Alex Lancaster?

The following additional context might be relevant in answering this question:

# Avery Lancaster

## Summary
- **Date of Birth**: March 15, 1985  
- **Job Title**: Co-Founder & Chief Executive Officer (CEO)  
- **Location**: San Francisco, California  

## Insurellm Career Progression
- **2015 - Present**: Co-Founder & CEO  
  Avery Lancaster co-founded Insurellm in 2015 and has since guided the company to its current position as a leading Insurance Tech provider. Avery is known for her innovative leadership strategies and risk management expertise that have catapulted the company into the mainstream insurance market.  

- **2013 - 2015**: Senior Product Manager at Innovate Insurance Solutions  
  Before launching Insurellm, Avery was a leading Senior Product Manager at Innovate Insurance Solutions, where she developed groundbreaking insurance products aimed at the tech sector.  

- **2010 - 2013**: Business Analyst at Edge Analytics  
  Prior to joining Innova

In [17]:
# Chat関数
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history
    message = add_context(message)
    messages.append({"role": "user", "content": message})

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

## GradioのChatインターフェイスを使用して、これを紹介します。
- LLMとのチャットをプロトタイプする迅速かつ簡単な方法
- 試しに、再度 `Who is Alex Lancaster?` とでも聞いてみる。
- すると、`正確には Avery Lancaster だ`みたいなことを言われる（適切！）。

In [18]:
view = gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7871
* To create a public link, set `share=True` in `launch()`.
